In [1]:
!pip install torch transformers datasets numpy pandas matplotlib seaborn tqdm -q

In [16]:
# 전체 노트북에서 사용할 라이브러리 임포트

# 딥러닝 모델 및 계산 관련
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, T5ForConditionalGeneration, T5Tokenizer
import numpy as np

# 데이터 처리 및 관리
import pandas as pd
from datasets import load_dataset

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 유틸리티
import random
from tqdm.auto import tqdm
import textwrap

# 시각화 스타일 설정 
sns.set_style("whitegrid")

try:
    plt.rc('font', family='NanumGothic') 
    plt.rc('axes', unicode_minus=False) # 마이너스 부호 깨짐 방지
except:
    print("나눔고딕 폰트를 찾을 수 없습니다. 기본 폰트로 설정됩니다. 그래프에 한글이 깨질 수 있습니다.")

In [17]:
# tldr_news(datasetnotfounderror) => cnn_dailymail
# article , highlight , id 로 구성됨
dataset = load_dataset("cnn_dailymail","3.0.0",split='train[:100]')

human_texts = [example['article'] for example in dataset]

print(f"Total Human gen text # : {len(human_texts)}")
print("\n--- Sample text ---")

wrapped_text = textwrap.fill(human_texts[0], width = 100)
print(wrapped_text)

Total Human gen text # : 100

--- Sample text ---
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20
million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a
spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the
disappointment of gossip columnists around the world, the young actor says he has no plans to
fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those
people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or
something similar," he told an Australian interviewer earlier this month. "I don't think I'll be
particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books
and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see
the horror film "Hostel: Part II," currently six places below his number on

In [18]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm
import textwrap

# 모델 이름 지정하기
model_name = "openai-community/gpt2"

# 토크나이저와 모델 불러오기
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 패딩 토큰 설정 (모델이 문장 길이를 맞출 때 사용)
tokenizer.pad_token = tokenizer.eos_token

# 장치 설정 (GPU 우선으로, 없으면 cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")


# AI 텍스트 생성을 위한 원본 데이터 
human_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is a field of computer science that focuses on creating intelligent machines.",
    "Hugging Face is a company that provides tools for building applications using machine learning."
]


# AI 텍스트 생성 루프
ai_texts = []
for text in tqdm(human_texts, desc="AI 텍스트 생성 중"):
    # 프롬프트로 사용할 앞 15 단어 추출
    prompt = " ".join(text.split()[:15])

    # 프롬프트 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(device) # <--- 'generator.tokenizer'가 아닌 'tokenizer' 사용

    # 텍스트 생성
    output = model.generate( 
        **inputs,
        max_length=150,
        do_sample=True, # more human-like
        top_k=50, #
        top_p=0.95,
        num_return_sequences=1
    )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    ai_texts.append(generated_text)

# 8. 결과 확인
print(f"\nTotal AI gen text #: {len(ai_texts)}")
print("\n--- Sample text ---")
# textwrap을 사용해 보기 좋게 출력
wrapped_text = textwrap.fill(ai_texts[0], width=100)
print(wrapped_text)

Using device: cpu


AI 텍스트 생성 중: 100%|██████████| 3/3 [00:13<00:00,  4.49s/it]


Total AI gen text #: 3

--- Sample text ---
The quick brown fox jumps over the lazy dog. The fox then runs to his home and looks for his
friends. They will be waiting as they sleep.  The fox picks up an apple, tosses it in his mouth and
slowly rolls over a dead tree.  He sits up and looks up from where he is waiting for his friends.
"Where's the apple?" he asks, turning around.  The old fox answers, looking disappointed and scared
at the fox. The old fox looks at him like he is angry. He laughs and looks down at the apple. He
does not have an answer.  The fox goes back to his normal life. He does not have any friends, but he
is always at


In [ ]:
# 1-3. Dataframe으로 합치기 
import pandas as pd
import numpy as np

# Label
human_labels = [0] * len(human_texts)
ai_labels = [1] * len(ai_texts)

# Combine 
all_texts = human_texts + ai_texts
all_labels = human_labels + ai_labels

# Pandas Dataframe
data = {
    'text': all_texts,
    'label' : all_labels
}
df = pd.DataFrame(data)

# Shuffle using frac = 1
df = df.sample(frac=1).reset_index(drop=True)

# Check
print("Data Generate Complete!!!!!!!!")
print(f"Total data : {len(df)}")

print("\n [Generated sample (top 5)]")
print(df.head())

print("\nDataset Info.")
df.info()



Data Generate Complete!!!!!!!!
Total data : 6

 [Generated sample (top 5)]
                                                text  label
0  Artificial intelligence is a field of computer...      1
1       The quick brown fox jumps over the lazy dog.      0
2  Hugging Face is a company that provides tools ...      0
3  Artificial intelligence is a field of computer...      0
4  Hugging Face is a company that provides tools ...      1

Dataset Info.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6 non-null      object
 1   label   6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


In [19]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# 사용할 모델의 토크나이저
# 'bert-base-cased': 영어 텍스트 분류
model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

#  데이터셋 into 훈련용, 검증용
train_df, val_df = train_test_split(
    df,
    test_size=0.2, # 전체 데이터셋의 20%가 val
    stratify=df['label'], #훈련/검증 데이터셋의 라벨 비율(0과 1의 비율)을 원본과 동일하게 유지
    random_state=42 # 난수 시드는 고정 for 재현
)

print("데이터 분할 완료!!!!!!!!!")
print(f"훈련 데이터 개수: {len(train_df)}개")
print(f"검증 데이터 개수: {len(val_df)}개")

# 텍스트 데이터를 토큰화하여 모델 입력 형태로 변환
# PyTorch 텐서 형태로 반환
train_encodings = tokenizer(
    list(train_df['text']),
    truncation=True,      # 문장이 최대 길이를 넘으면 자르기
    padding=True,         # 문장 길이를 맞추기 위해 패딩 추가
    max_length=128,       # 최대 길이 설정
    return_tensors='pt'   # 파이토치 텐서로 반환
)

val_encodings = tokenizer(
    list(val_df['text']),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

# 5. 토큰화 결과 확인
print("\n텍스트 토큰화 완료!!!!!!!!!!")
print("\n[토큰화된 훈련 데이터 정보]")
# input_ids: 토큰의 숫자 ID
# attention_mask: 패딩된 부분은 모델이 무시하도록 알려주는 역할
print(train_encodings.keys())
print("\n[첫 번째 훈련 데이터의 input_ids (일부)]")
print(train_encodings['input_ids'][0][:20]) # 첫 번째 문장의 앞 20개 토큰 ID 출력

데이터 분할 완료!!!!!!!!!
훈련 데이터 개수: 4개
검증 데이터 개수: 2개

텍스트 토큰화 완료!!!!!!!!!!

[토큰화된 훈련 데이터 정보]
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

[첫 번째 훈련 데이터의 input_ids (일부)]
tensor([  101, 20164, 10932, 10289,  1110,   170,  1419,  1115,  2790,  5537,
         1111,  1459,  4683,  1606,  3395,  3776,   119,   102,     0,     0])


In [20]:
import torch

# PyTorch를 위한 커스텀 Dataset 클래스 정의
class AITextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # encodings 딕셔너리의 각 값(텐서)에 대해 idx에 해당하는 슬라이스를 가져옴
        item = {key: val[idx] for key, val in self.encodings.items()}
        # 해당 아이템의 레이블을 텐서 형태로 추가
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        # 데이터셋의 총 길이 반환
        return len(self.labels)

# 훈련 및 검증 데이터셋 생성
# pandas의 iloc을 사용해 라벨 리스트를 올바르게 가져온다.
train_labels = train_df['label'].iloc[:].tolist()
val_labels = val_df['label'].iloc[:].tolist()

train_dataset = AITextDataset(train_encodings, train_labels)
val_dataset = AITextDataset(val_encodings, val_labels)

# DataLoader 생성
from torch.utils.data import DataLoader

# batch_size는 모델이 한 번에 학습할 데이터의 양
# 작은 데이터셋이므로 2로 설정하기
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# DataLoader 작동 확인
print("DataLoader 생성 완료!!!")
print("훈련 데이터 로더에서 배치 하나씩 꺼내보기")

# iter와 next를 사용해 데이터 로더에서 첫 번째 배치를 꺼냄
data_batch = next(iter(train_loader))

print("\n[배치의 Key 확인]")
print(data_batch.keys())

print("\n[배치의 Shape 확인]")
# input_ids의 shape: [배치 사이즈, 최대 문장 길이]
print("Input IDs shape:", data_batch['input_ids'].shape)
# labels의 shape: [배치 사이즈]
print("Labels shape:", data_batch['labels'].shape)

DataLoader 생성 완료!!!
훈련 데이터 로더에서 배치 하나씩 꺼내보기

[배치의 Key 확인]
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

[배치의 Shape 확인]
Input IDs shape: torch.Size([2, 128])
Labels shape: torch.Size([2])


In [21]:
# 필요한 라이브러리 가져오기
from transformers import BertForSequenceClassification, AdamW

# 분류를 위한 BERT 모델 불러오기
# model_name은 이전에 토크나이저를 불러올 때 사용한 이름과 동일해야 함
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2 #사람(0), AI(1)
)

# 모델을 GPU 또는 CPU 장치로 이동
model.to(device)

# 옵티마이저 설정
# model.parameters()는 모델이 학습할 모든 파라미터(가중치)를 의미합니다.
# lr = learning rate = alpha
optimizer = AdamW(model.parameters(), lr=5e-5)

# 5. 설정 완료 확인
print("------------------------")
print("모델 및 옵티마이저 설정이 완료!!!!!!!!!!")   
print(f"사용 모델: {model_name}")
print(f"라벨 개수: {model.config.num_labels}개")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


------------------------
모델 및 옵티마이저 설정이 완료!!!!!!!!!!
사용 모델: bert-base-cased
라벨 개수: 2개


c:\Users\SeungWoo\Anaconda3\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
# 1. 필요한 라이브러리 추가
from tqdm import tqdm # 학습 진행 상황을 보여주는 라이브러리
import numpy as np

# 2. 훈련 루프 설정
# 에폭(Epoch): 전체 데이터를 몇 번 반복해서 학습할지 결정합니다.
epochs = 3

for epoch in range(epochs):
    print(f"--- Epoch {epoch + 1}/{epochs} ---")

    # --- 훈련 단계 (Training) ---
    model.train() # 모델을 훈련 모드로 설정
    total_train_loss = 0
    for batch in tqdm(train_loader, desc="Training"):
        # 이전 배치의 그래디언트 초기화
        optimizer.zero_grad()

        # 데이터를 device로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델에 데이터 입력
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # 손실(loss) 계산
        loss = outputs.loss
        total_train_loss += loss.item()

        # 역전파 (손실을 기반으로 그래디언트 계산)
        loss.backward()
        # 파라미터 업데이트
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"평균 훈련 손실 (Avg Train Loss): {avg_train_loss:.4f}")

    # --- 검증 단계 (Validation) ---
    model.eval() # 모델을 평가 모드로 설정
    total_val_loss = 0
    correct_predictions = 0
    with torch.no_grad(): # 그래디언트 계산 비활성화
        for batch in tqdm(val_loader, desc="Validation"):
            # 데이터를 device로 이동
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 모델에 데이터 입력
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            # 손실 및 예측 결과(logits) 가져오기
            loss = outputs.loss
            logits = outputs.logits
            total_val_loss += loss.item()

            # 정확도 계산
            preds = torch.argmax(logits, dim=1)
            correct_predictions += (preds == labels).sum().item()

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = correct_predictions / len(val_dataset)
    print(f"평균 검증 손실 (Avg Val Loss): {avg_val_loss:.4f}")
    print(f"검증 정확도 (Val Accuracy): {val_accuracy:.4f}")

print("\n훈련이 완료!")

--- Epoch 1/3 ---


Training: 100%|██████████| 2/2 [00:03<00:00,  1.66s/it]


평균 훈련 손실 (Avg Train Loss): 0.8032


Validation: 100%|██████████| 1/1 [00:00<00:00,  3.74it/s]


평균 검증 손실 (Avg Val Loss): 0.6519
검증 정확도 (Val Accuracy): 0.5000
--- Epoch 2/3 ---


...

평균 검증 손실 (Avg Val Loss): 0.5528
검증 정확도 (Val Accuracy): 1.0000
--- Epoch 3/3 ---


Training: 100%|██████████| 2/2 [00:02<00:00,  1.06s/it]


평균 훈련 손실 (Avg Train Loss): 0.5621


Validation: 100%|██████████| 1/1 [00:00<00:00,  3.91it/s]

평균 검증 손실 (Avg Val Loss): 0.5395
검증 정확도 (Val Accuracy): 1.0000

훈련이 완료!


In [23]:
# 1. 예측 결과를 해석하기 위한 라벨 맵 정의
label_map = {0: "사람이 작성한 글 (Human)", 1: "AI가 생성한 글 (AI)"}

# 2. 예측 함수 정의
def predict(text):
    # 모델을 평가 모드로 설정
    model.eval()

    # 입력된 텍스트를 토큰화
    inputs = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt' # 파이토치 텐서로 반환
    )

    # 모든 텐서를 device로 이동
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # 그래디언트 계산 비활성화
    with torch.no_grad():
        # 모델에 입력을 전달하여 예측 수행
        outputs = model(**inputs)

    # 모델의 출력(logits)에서 가장 높은 값의 인덱스를 예측 라벨로 선택
    predicted_label_id = torch.argmax(outputs.logits, dim=1).item()

    # 라벨 ID를 실제 라벨 이름으로 변환하여 반환
    return label_map[predicted_label_id]


# 3. 새로운 문장으로 예측 테스트
test_sentence_1 = "I just came back from my trip to Jeju island. The weather was amazing and the food was delicious."
test_sentence_2 = "The integration of artificial intelligence into daily life has accelerated transformative changes across various sectors."

prediction_1 = predict(test_sentence_1)
prediction_2 = predict(test_sentence_2)

print("\n--- 새로운 문장 예측 테스트 ---")
print(f"입력 문장 1: \"{test_sentence_1}\"")
print(f"모델 예측 : {prediction_1}\n")

print(f"입력 문장 2: \"{test_sentence_2}\"")
print(f"모델 예측 : {prediction_2}")


--- 새로운 문장 예측 테스트 ---
입력 문장 1: "I just came back from my trip to Jeju island. The weather was amazing and the food was delicious."
모델 예측 : 사람이 작성한 글 (Human)

입력 문장 2: "The integration of artificial intelligence into daily life has accelerated transformative changes across various sectors."
모델 예측 : 사람이 작성한 글 (Human)


In [24]:
# 1. 필요한 라이브러리 추가
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
from sklearn.metrics import accuracy_score, classification_report

# --------------------------------------------------------------------------
# 1단계: 더 큰 테스트 데이터셋 생성 (총 100개)
# --------------------------------------------------------------------------
print(" 1단계: 100개의 테스트 데이터셋을 생성합니다...")

# 50개의 새로운 '사람' 문장 샘플 (필요에 따라 더 추가하거나 변경 가능)
test_human_texts = [
    "I'm planning a vacation to Italy next month.",
    "This new coffee shop has the best espresso I've ever tasted.",
    "Yesterday, I spent the entire afternoon reading a book in the park.",
    "My favorite movie of all time is The Shawshank Redemption.",
    "Learning a new language can be challenging, but it's also very rewarding.",
    "I need to go grocery shopping after work today.",
    "The final episode of the series was a bit disappointing.",
    "My computer is running so slow, I think it's time for an upgrade.",
    "She has been practicing the piano every day for the past five years.",
    "We went hiking in the mountains and the view from the top was breathtaking.",
    "I'm not feeling well, so I might take a sick day tomorrow.",
    "The traffic was terrible this morning on my way to the office.",
    "He explained the complex topic in a very simple and understandable way.",
    "I finally finished the puzzle that I've been working on for a week.",
    "It's important to get enough sleep for good health.",
    "The museum has a new exhibition on ancient Egyptian artifacts.",
    "She sent me a postcard from her trip to Paris.",
    "I'm trying to eat healthier by including more vegetables in my diet.",
    "He is a talented artist who paints beautiful landscapes.",
    "I accidentally deleted an important file from my computer.",
    "The weather forecast predicts rain for this weekend.",
    "She volunteers at the local animal shelter every Saturday.",
    "I'm looking forward to the concert next Friday.",
    "He always makes me laugh with his great sense of humor.",
    "I have a dentist appointment scheduled for Tuesday afternoon.",
    "This recipe requires a few ingredients that I don't have.",
    "The library is a quiet place to study and concentrate.",
    "I need to remember to charge my phone before I go to bed.",
    "They are renovating their house, so it's a bit messy right now.",
    "I enjoy listening to classical music while I work.",
    "The history of this city is fascinating and full of interesting stories.",
    "I'm taking a course to improve my public speaking skills.",
    "She is known for her generosity and kindness to others.",
    "I misplaced my keys and I can't find them anywhere.",
    "The park is a great place for a picnic on a sunny day.",
    "He is allergic to peanuts, so we need to be careful with the food.",
    "I'm trying to reduce my screen time and read more books.",
    "The new restaurant downtown has received excellent reviews.",
    "She has a remarkable talent for learning new languages quickly.",
    "I need to do my laundry this evening.",
    "The view of the sunset from the beach was absolutely stunning.",
    "He is training for a marathon, so he runs every morning.",
    "I'm saving money to buy a new car next year.",
    "The customer service representative was very helpful and polite.",
    "I love the smell of freshly baked bread.",
    "She adopted a cute puppy from the animal shelter.",
    "I have to prepare a presentation for the meeting tomorrow.",
    "The documentary about wildlife was both educational and entertaining.",
    "He fixed the broken chair with some glue and a few screws.",
    "I'm excited to see my family during the holidays."
]


# AI 텍스트 생성을 위한 오리지널 GPT-2 모델 파이프라인 다시 로드
# (우리가 훈련시킨 모델이 아닌, 초기 데이터 생성용 모델)
text_generator = pipeline("text-generation", model="openai-community/gpt2", device=device)

# 50개의 'AI' 문장 생성
test_ai_texts = []
for text in tqdm(test_human_texts, desc="AI 테스트 텍스트 생성 중"):
    prompt = " ".join(text.split()[:15]) # 처음 15단어를 프롬프트로 사용
    generated_outputs = text_generator(prompt, max_length=len(text.split()) + 20, num_return_sequences=1)
    generated_text = generated_outputs[0]['generated_text']
    test_ai_texts.append(generated_text)

# 데이터프레임으로 결합
human_df_test = pd.DataFrame({'text': test_human_texts, 'label': 0})
ai_df_test = pd.DataFrame({'text': test_ai_texts, 'label': 1})
test_df = pd.concat([human_df_test, ai_df_test], ignore_index=True)
test_df = test_df.sample(frac=1).reset_index(drop=True) # 데이터 섞기

print(f" 총 {len(test_df)}개의 테스트 데이터셋 생성을 완료했습니다.")


# --------------------------------------------------------------------------
# 2단계: 훈련된 모델로 전체 테스트 데이터셋 추론
# --------------------------------------------------------------------------
print("\n 2단계: 훈련된 모델로 전체 데이터에 대한 추론을 시작합니다...")

predictions = []
# test_df의 각 텍스트에 대해 예측 수행
for text in tqdm(test_df['text'], desc="추론 진행 중"):
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    pred_label_id = torch.argmax(outputs.logits, dim=1).item()
    predictions.append(pred_label_id)

# 예측 결과를 데이터프레임에 추가
test_df['predicted_label'] = predictions
print("추론을 완료했습니다.")


# --------------------------------------------------------------------------
# 3단계: 정확도 계산 및 결과 확인
# --------------------------------------------------------------------------
print("\n3단계: 최종 성능 평가 결과를 보여드립니다...")

# 정확도 계산
accuracy = accuracy_score(test_df['label'], test_df['predicted_label'])

# 상세 리포트 생성
report = classification_report(test_df['label'], test_df['predicted_label'], target_names=['Human (class 0)', 'AI (class 1)'])

print("\n--- 최종 모델 평가 결과 ---")
print(f"Total accuracy: {accuracy * 100:.2f}%")
print("\n[상세 분류 리포트]")
print(report)

print("\n[예측 결과 샘플 (상위 10개)]")
# 실제 정답(label)과 모델의 예측(predicted_label)을 비교해볼 수 있도록 출력
print(test_df.head(10))

# 참고: 현재 모델은 단 6개의 데이터로만 훈련되었기 때문에,정확도가 낮음
# 이 코드는 '평가 파이프라인'이 잘 작동하는지를 확인하는 데 의의가 있습니다.

➡️ 1단계: 100개의 테스트 데이터셋을 생성합니다...


AI 테스트 텍스트 생성 중:   0%|          | 0/50 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
AI 테스트 텍스트 생성 중: 100%|██████████| 50/50 [00:26<00:00,  1.89it/s]


 총 100개의 테스트 데이터셋 생성을 완료했습니다.

 2단계: 훈련된 모델로 전체 데이터에 대한 추론을 시작합니다...


추론 진행 중: 100%|██████████| 100/100 [00:05<00:00, 18.35it/s]

추론을 완료했습니다.

3단계: 최종 성능 평가 결과를 보여드립니다...

--- 최종 모델 평가 결과 ---
Total accuracy: 60.00%

[상세 분류 리포트]
                 precision    recall  f1-score   support

Human (class 0)       0.56      1.00      0.71        50
   AI (class 1)       1.00      0.20      0.33        50

       accuracy                           0.60       100
      macro avg       0.78      0.60      0.52       100
   weighted avg       0.78      0.60      0.52       100


[예측 결과 샘플 (상위 10개)]
                                                text  label  predicted_label
0  I have a dentist appointment scheduled for Tue...      0                0
1  I'm trying to eat healthier by including more ...      0                0
2  Yesterday, I spent the entire afternoon readin...      1                0
3  I'm not feeling well, so I might take a sick d...      0                0
4  I'm excited to see my family during the holida...      1                0
5  She volunteers at the local animal shelter eve...      0               